## 创建深度的DNN模型

### 基础库和函数

这里需要导入的库有：

- numpy：这个不用介绍
- h5py 用来分析H5 文件的包.
- matplotlib 不用介绍
- dnn_utils：作者给定的一些必要的函数：sigmoid, sigmoid_backward, relu, relu_backward
- testCases 测试函数
- np.random.seed(1) 保证随机数的确定性



In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases_v3 import *
from dnn_utils_v2 import sigmoid, sigmoid_backward, relu, relu_backward

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

In [ ]:
def sigmoid(Z):
    """
    输出：
    A: 返回最终计算的值
    cache:跟A一样的值，用来反向传播的使用
    """
    
    A = 1.0/(1.0+np.exp(-Z))
    cache = Z
    
    return A,cache

def sigmoid_backward(dA,cache):
    """
    dA:表示后一层的梯度值
    cache:表示之前保存的前一层的缓存的值
    """
    
    # 这里将前一层缓存的值直接代入
    Z = cache
    
    s = 1.0/(1.0+np.exp(-Z))
    dZ = dA * s * (1-s)
    
    assert(dZ.shape == Z.shape)
    
    return dZ


In [ ]:
def relu(Z):
    """
    输出：
    A: 返回前向计算的值
    cache:跟A一样的值，用来反向传播的使用
    """
    
    # 0与Z比较
    A = np.maximum(0,Z)
    
    assert(A.shape == Z.shape)
    
    # 缓存该值用于反向传播计算
    cache = Z
    
    return A,cache

def relu_backward(dA,cache):
    """
    dZ: 最终计算的梯度值
    """
    Z = cache
    dZ = np.array(dA, copy=True) # just converting dz to a correct object.

    # When z <= 0, you should set dz to 0 as well. 
    dZ[Z <= 0] = 0

    assert (dZ.shape == Z.shape)

    return dZ

    

### 基础辅助函数的定义

这里我们需要定义一些基础的辅助函数，这样是为了后面实现一个更具深度的模型建立。主要的建立的函数如下：

- 参数初始化函数：2层网络与L层网络
- 前向网络传播函数
    - 线性传播层WX+b
    - 激活函数：relu/sigmoid
    - 组合上面[线性+激活]层的前向传播
    - 堆叠[线性+relu激活]层的前向传播以及[线性+sigmoid激活]
- loss 函数
- 反向传播函数
    - 线性反向传播层
    - 激活函数的反向传播：relu/sigmoid
    - 组合上面[线性+激活]层的反向传播
    - 堆叠[线性+relu激活]层的反向传播以及[线性+sigmoid激活]
 - 参数更新函数
 
 上面的实践函数的过程如下图：
 ![](imgs/1.jpg)
 
 下面让我开始码代码吧。
 
 ### 参数初始化
 
 #### 2层网络的初始化
 
 我们需要了解一下内容：
 - 模型的结果是：线性->relu激活函数->线性->sigmoid激活函数
 - W权重参数需要用到的numpy的函数是：np.random.randn(shape)*0.01
     -  W权重参数可以使用符合高斯分布的随机数进行初始化：np.random.randn()*0.01
 - b偏置值参数直接复制为0
 

In [4]:
def initialize_parameters(n_x, n_h, n_y):
    """
    输入：
        n_x: 表示输入的数据的个数
        n_h: 中间隐藏层的神经元的个数
        n_y: 表示输出的数据的个数
    """
    
    np.random.seed(1)
    # W1的shape是根据前一层和后一层的维度确定
    W1 = np.random.randn(n_h,n_x) * 0.01
    # b的维度是根据当前层的维度来确定
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y,n_h) * 0.01
    b2 = np.zeros((n_y,1))
    
    # 辅助判断参数的shape
    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))
    
    parameters={"W1":W1,
               "b1":b1,
               "W2":W2,
               "b2":b2}
    
    return parameters

In [5]:
#测试函数
parameters = initialize_parameters(3,2,1)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))


W1 = [[ 0.01624345 -0.00611756 -0.00528172]
 [-0.01072969  0.00865408 -0.02301539]]
b1 = [[0.]
 [0.]]
W2 = [[ 0.01744812 -0.00761207]]
b2 = [[0.]]


#### L层网络的初始化

如果需要对多层网络的初始化，这里我们就需要每层网络的隐藏网络神经元的个数。下面的图标中揭示如何计算权重与偏置的shape:
![](imgs/2.jpg)

上图中$n^l$就是第l层的神经元的个数。

这里有一个例子关于WX+b的结果如下图：
![](imgs/3.jpg)
![](imgs/4.jpg)

关于实践L层网络的coding我们要注意点：
- 输入的一个数组layer_dims[2,4,1]
    - 2: 表示第0层的神经元的个数，4:....
    - W1(4,2),W2(1,4)
    - b1(4,1),b2(1,1)
    
具体的coding如下


In [8]:
def initialize_parameters_deep(layer_dims):
    """
    输入：
        layer_dims：是各个层的神经元的个数组成的数组
    """
    np.random.seed(3)
    parameters = {}
    # 获得数组的程度
    L = len(layer_dims)
    
    # 从第二层开始
    for l in range(1,L):
        # 为了获得前一层的维度，需要减1
        parameters['W'+str(l)] = np.random.randn(
            layer_dims[l],layer_dims[l-1])*0.01
        parameters['b'+str(l)] = np.zeros((
            layer_dims[l],1))
        
    return parameters
    

In [9]:
parameters = initialize_parameters_deep([5,4,3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[0.]
 [0.]
 [0.]]


### 前向传播模型

这里我们需要完成以下的基础函数：
- LINEAR
- LINEAR -> ACTIVATION ACTIVATION 要么是 ReLU 要么是 Sigmoid.
- [LINEAR -> RELU] ×× (L-1) -> LINEAR -> SIGMOID (whole model)


#### 线性前向传播

这里的线性前向传播就是简单的权重与偏置值的相乘与相加。也就是如下图

这里有一个例子关于WX+b的结果如下图：
![](imgs/3.jpg)
![](imgs/4.jpg)

但是为了表示更加一般的表示，我们一般使用如下的方式：
![](imgs/5.jpg)

主要注意的是：
- 输入层的X,我们用$A^{[0]}$表示
- WX:这里 使用的np.dot来实践

具体的coding如下：


In [2]:
def linear_forward(A, W, b):
    """
    输入：
        A:表示前一层的计算结果
        W:表示当前层的权重
        b:表示当前层的偏置
        
    输出：
        Z: 计算结果
        cache:A,W,b的数据
    """
    Z = np.dot(W,A) + b
    
    cache = (A,W,b)
    
    return Z,cache

In [11]:
A, W, b = linear_forward_test_case()

Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

Z = [[ 3.26295337 -1.23429987]]


#### 线性 - 激活函数的前向传播

- Sigmoid激活函数：
    - ”a“表示是激活函数的a
    - "cache"表示缓存的上一层的输入值
![](imgs/6.jpg)

- RELU激活函数：
![](imgs/7.jpg)

这里是将线性 - 激活函数的组合，所以我们需要使用上面的线性函数。计算过程如下：


In [3]:
def linear_activation_forward(A_prev, W, b, activation):
    """
    输入：
    A_prev:表示上一层的激活函数计算的值或者输入层数据
    W,b:表述参数
    activation： 表示当前激活函数样式："sigmoid"&"relu"
    
    输出：
    A:当前层的输出
    cache:缓存linear_cache, activation_cache
    """
    
    if activation == "sigmoid":
        Z,linear_cache = linear_forward(A_prev,W,b)
        A,activation_cache = sigmoid(Z)
    elif activation == "relu":
        Z,linear_cache = linear_forward(A_prev,W,b)
        A,activation_cache = relu(Z)
    
    cache = (linear_cache, activation_cache)
    return A,cache
    

In [13]:
A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))


With sigmoid: A = [[0.96890023 0.11013289]]
With ReLU: A = [[3.43896131 0.        ]]


#### L 层模型前向传播

这里的L层网络的前向传播，其实就是组合层一个模型，让数据从输入自然的经过各种组合最后输出。具体的过程如下图：
![](imgs/8.jpg)

从上图中我们可以看出：
- AL是一直计算获得，前L-1获得
- ![](imgs/9.jpg)
- 同时我们这里仅仅需要对这个组合[LINEAR->RELU]进行(L-1)次
具体的coding如下



In [4]:
def L_model_forward(X, parameters):
    """
    输入：
        X: 输入的数据
        parameters:具体的参数，
    """
    
    caches = []
    A = X
    # 通过参数就可以获得模型的层数
    L = len(parameters) // 2
    
    # 前L-1层的网络
    for l in range(1,L):
        A_prev = A
        A,cache = linear_activation_forward(A_prev,
                            parameters['W'+str(l)],
                            parameters['b'+str(l)],
                                           'relu')
        caches.append(cache)
        
    # 最后一层的网络   
    AL,cache = linear_activation_forward(A,
                            parameters['W'+str(L)],
                            parameters['b'+str(L)],
                                        'sigmoid')
    caches.append(cache)
    
    return AL, caches

In [5]:
X, parameters = L_model_forward_test_case_2hidden()
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))


AL = [[0.03921668 0.70498921 0.19734387 0.04728177]]
Length of caches list = 3


### Loss函数

使用cross-entropy来计算损失函数：
![](imgs/10.jpg)


In [6]:
def compute_cost(AL,Y):
    """
    输入：
        AL:最后一层的输出值
        Y:标签数据
    """
    
    m = Y.shape[1]
    # j计算loss值
    cost = -np.sum(np.muliply(np.log(AL),Y)+
                   np.multiply(np.log(1-AL),1-Y))
    
    cost = np.squeeze(cost)
    
    return cost

### 反向传播模型

反向传播为了将loss传递到每一层的参数，从而更新参数的数据。具体的步骤如下图所示：
![](imgs/11.jpg)

这里有一个比较关键的反向传播过程，那就是第一层的dz1的计算：
![](imgs/12.jpg)

下面我们同前向传播一样，我们需要做创建如下的基础函数：
- LINEAR 反向传播
- LINEAR -> ACTIVATION 反向传播，relu/sigmoid
- [LINEAR -> RELU] ×× (L-1) -> LINEAR -> SIGMOID 反向传播

#### 线性的反向传播函数

假设第l层，其线性前向传播如下:
![](imgs/5.jpg)

需要计算的示意图如下：
![](imgs/13.jpg)

具体的计算公式如下：
![](imgs/14.jpg)

如下coding


In [2]:
def linear_backward(dZ, cache):
    """
    输入：
        dZ 前一层的Activate导数
        cache: 该层缓存的（A_prev,W,b）
    
    输出：
        dA_prev：loss的梯度
        dW:W的梯度
        db:b的梯度
    """
    A_prev,W,b = cache
    m = A_prev.shape[1]
    
    dW = np.dot(dZ,A_prev.T)/m
    db = np.sum(dZ,axis=1,keepdims=True)/m
    dA_prev = np.dot(W.T,dZ)
    
    return dA_prev,dW,db
    

In [3]:
dZ, linear_cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))


dA_prev = [[ 0.51822968 -0.19517421]
 [-0.40506361  0.15255393]
 [ 2.37496825 -0.89445391]]
dW = [[-0.10076895  1.40685096  1.64992505]]
db = [[1.01258895]]


#### 线性-激活函数的反向传播

这是需要跟刚刚的线性反向传播组合在一起使用，所以我们需要做获得sigmoid_backward 与 relu_backward函数组合。
这里需要使用的计算公式是：
![](imgs/15.jpg)

具体的coding如下：


In [5]:
def linear_activation_backward(dA,cache,activation):
    """
    输入：
        dA: 前一层的梯度
        cache: 前向传播缓存的值：linear_cache,activation_catche
        activation: 激活函数的形式
    """
    
    linear_cache, activation_cache = cache
    
    
    if activation == 'relu':
        dZ = relu_backward(dA,activation_cache)
        dA_prev,dW,db = linear_backward(dZ,linear_cache)
        
    elif activation == 'sigmoid':
        dZ = sigmoid_backward(dA,activation_cache)
        dA_prev,dW,db = linear_backward(dZ,linear_cache)
        
    return dA_prev,dW,db

In [6]:
AL, linear_activation_cache = linear_activation_backward_test_case()

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))


sigmoid:
dA_prev = [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
db = [[-0.11459244]]

relu:
dA_prev = [[ 0.44090989 -0.        ]
 [ 0.37883606 -0.        ]
 [-0.2298228   0.        ]]
dW = [[ 0.44513824  0.37371418 -0.10478989]]
db = [[-0.41675785]]


#### L 层网络的反向传播

关于L层网络的反向传播，是对整个网络的组合在一起，再进行梯度训练。所以这里我们需要时刻要注意的是需要利用之前前向传播缓存的数据(X,W,b,z)，然后再迭代完成每一层的梯度计算。具体的流程如下图：
![](imgs/16.jpg)

**loss函数的梯度计算**  

从上面的图中我们这知道反向传播是从最后一层开始的，也就是计算loss值后开始计算的。所以需要计算loss值的反向传播。
线拿到loss值得函数：
![](imgs/10.jpg)

计算上面得dAL得值也比较简单：  
`dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))`  

**最后一层linear-activation的计算**  
从上面的图中我们可以看到，在获得output层的梯度dAL后，这里我们需要需要计算的是一个linear-sigmoid层。  

**循环层linear-relu的计算**  
后面的数据都是关于linear-relu循环层的计算，所以比较简单。当然每个层计算的梯度都需要按一定的格式记录保存。    
$grads[^"dW^"+str(l)]=dW^{[l]}$  

- l 表示第几层
- grads表示是一个字典数据



In [11]:
def L_model_backward(AL, Y, caches):
    
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    # 这里计算loss function的梯度
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    # 计算最后一层linear-sigmoid的梯度
    current_cache = caches[L-1]
    grads["dA"+str(L)],grads["dW"+str(L)],grads["db"+str(L)]=linear_activation_backward(dAL, 
                                                                                        current_cache,"sigmoid")
    
    # 迭代计算其他层linear-relu的梯度
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        grads["dA"+str(l+1)],grads["dW"+str(l+1)],grads["db"+str(l+1)]=linear_activation_backward(grads["dA"+str(l+2)],
                                                                                            current_cache,"relu")
             
    return grads
    

In [15]:
AL, Y_assess, caches = L_model_backward_test_case()
grads = L_model_backward(AL, Y_assess, caches)
print_grads(grads)

dW1 = [[0.41010002 0.07807203 0.13798444 0.10502167]
 [0.         0.         0.         0.        ]
 [0.05283652 0.01005865 0.01777766 0.0135308 ]]
db1 = [[-0.44014127]
 [ 0.        ]
 [-0.05670698]]
dA1 = [[ 0.12913162 -0.44014127]
 [-0.14175655  0.48317296]
 [ 0.01663708 -0.05670698]]


#### 参数更新
参数更新的公式如下：
![](imgs/17.jpg)

这里只需要设置一个超参数$\alpha$值。需要主要的是，我们需要通过迭代对每一层的参数进行更新。具体的coding如下：

In [18]:
def update_parameters(parameters, grads, learning_rate):
    
    L = len(parameters)//2
    
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
        
    return parameters

In [19]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)

print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))


W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]
